<a href="https://colab.research.google.com/github/KIM-CHAEYOUNG/PROJECT_bike_station/blob/master/01_%EC%9A%B4%EC%98%81%EC%9D%B4%EB%A0%A5_%EB%8C%80%EC%97%AC%EB%B0%98%EB%82%A9%EC%8B%9C%EA%B0%84%EB%8C%80%EB%B3%84_%EB%B9%88%EB%8F%84_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## '01_운영이력자전거 사용 시간'을 통해 추출할 수 있을 데이터 예측2

1. 고양시 일대가 배드타운인 점을 고려할 때

    *자전거스테이션 이용과 막차 이후 사이에 상관관계가 존재할 것*

In [1]:
# 데이터 읽어오기
# 구글 클라우드 연동

from google.colab import drive

ROOT="/content/drive"
print(ROOT)
drive.mount(ROOT)

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
%cd drive/"My Drive"/"고양시"/

/content/drive/My Drive/고양시


In [4]:
%ls

'미래 영향요인 조사'/
 화성시/
 분석완료/
 01.운영이력.csv
 02.자전거스테이션.csv
 03.꽃박람회일정.csv
 03_map/
 03_map.zip
 04.KINTEX행사일정.csv
 05.기상정보.csv
'06.인구(거주)분포도(100M_X_100M).geojson'
 07.인구통계.csv
'08.행정경계(시군구).geojson'
'09.행정경계(읍면동).geojson'
 아이디어1/
'10.도시계획(공간시설).geojson'
'11.도시계획(공공문화체육시설).geojson'
'12.도시계획(교통시설).geojson'
'13.용도지역지구(습지보호지역).geojson'
 14.고양시_지적도.geojson
 15.도로명주소_건물.geojson
 16.도로명주소_도로.geojson
'17.일반건물_분포도(100M_X_100M).geojson'
 18.행사장_공간정보.csv
 19.전철역_공간정보.csv
 아이디어2/
 20.고양시_버스정류소.csv
 20.고양시_버스정류소.csv.20.고양시_버스정류소.vrt
 21.버스_정류장별_승하차_정보.csv
 22.주차장정보.csv
 23.고양시덕양구_DEM.img
 24.고양시일산동구_DEM.img
 25.고양시일산서구_DEM.img
'26.고양시 공연장 박물관 정보.csv'
'27.고양시 체육시설 현

In [5]:
# 패키지 설치
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import folium

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# '01_운영이력' 파일 읽어오기

In [6]:
bicycle_now=pd.read_csv("01.운영이력.csv")
bicycle_now

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0
3,15945544,2,2017-01-01 00:02:09,121,17,2017-01-01 00:15:58,133,14,15490.0,12,183971,0.0,1A844000000731,NaN
4,15945545,2,2017-01-01 00:03:32,320,29,2017-01-01 00:18:44,259,27,0.0,12,167475,0.0,1A84400000F343,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3142683,21718608,2,2019-12-31 23:52:39,222,16,2019-12-31 23:58:56,219,24,0.0,3,236957,0.0,1A84400000F881,NaN
3142684,21718610,2,2019-12-31 23:53:59,343,14,2019-12-31 23:57:59,311,15,0.0,99,0,4124665.0,1A84400000F9DA,NaN
3142685,21718611,2,2019-12-31 23:55:27,333,17,2020-01-01 00:06:24,322,5,0.0,99,0,4124667.0,1A84400000048B,NaN
3142686,21718613,2,2019-12-31 23:59:14,264,4,2020-01-01 00:10:39,210,13,0.0,12,180671,0.0,1A84400000F804,NaN


#***오전 12시-3시(막차 이후)*** 자전거 현황

In [7]:
# 새로운 칼럼('tempDATE') 추가
# 시/분/초를 쪼개서 시간값만 추출

bicycle_now['tempDATE']=bicycle_now.LEAS_DATE.apply(lambda x: x.split())

In [8]:
#시간값만 모아 컬럼['LEAS_hour']에 추가

bicycle_now['LEAS_hour']=bicycle_now.tempDATE.apply(lambda x:x[1].split(":")[0])

In [9]:
bicycle_now['LEAS_hour']

0          00
1          00
2          00
3          00
4          00
           ..
3142683    23
3142684    23
3142685    23
3142686    23
3142687    23
Name: LEAS_hour, Length: 3142688, dtype: object

In [10]:
#막차 시간 별 시리즈 생성

makcha0=bicycle_now['LEAS_hour']=='00'
makcha1=bicycle_now['LEAS_hour']=='01'
makcha2=bicycle_now['LEAS_hour']=='02'
makcha3=bicycle_now['LEAS_hour']=='03'

In [12]:
# 해당 시리즈를 bicycle_now의 칼럼으로 추가
makcha0=bicycle_now[makcha0]
makcha1=bicycle_now[makcha1]
makcha2=bicycle_now[makcha2]
makcha3=bicycle_now[makcha3]

In [13]:
# 해당 시리즈를 bicycle_now의 칼럼으로 추가
bicycle_makcha=makcha0.append([makcha1, makcha2, makcha3])
bicycle_makcha.head(3)

,LEAS_NO,LEAS_STAT,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO,RTN_DATE,RTN_STATION,RTN_DEF_NO,TRNV_QTY,MEMB_DIV,MEMB_NO,TEMP_MEMB_NO,BIKE_TAG,RTN_PROCESS,tempDATE,LEAS_hour
0,15945541,2,2017-01-01 00:00:41,213,18,2017-01-01 00:13:52,260,17,0.0,6,164203,0.0,1A844000000BB7,NaN,"[2017-01-01, 00:00:41]",00
1,15945542,2,2017-01-01 00:01:03,231,4,2017-01-01 00:50:24,231,17,31039.0,1,187551,0.0,1A844000000494,NaN,"[2017-01-01, 00:01:03]",00
2,15945543,2,2017-01-01 00:01:50,119,14,2017-01-01 01:01:50,0,0,NaN,12,168994,0.0,1A844000000533,1.0,"[2017-01-01, 00:01:50]",00


In [14]:
bicycle_makcha.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259527 entries, 0 to 3141705
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   LEAS_NO       259527 non-null  int64  
 1   LEAS_STAT     259527 non-null  int64  
 2   LEAS_DATE     259527 non-null  object 
 3   LEAS_STATION  259527 non-null  int64  
 4   LEAS_DEF_NO   259527 non-null  int64  
 5   RTN_DATE      259527 non-null  object 
 6   RTN_STATION   259527 non-null  int64  
 7   RTN_DEF_NO    259527 non-null  int64  
 8   TRNV_QTY      221944 non-null  float64
 9   MEMB_DIV      259527 non-null  object 
 10  MEMB_NO       259527 non-null  int64  
 11  TEMP_MEMB_NO  259527 non-null  float64
 12  BIKE_TAG      259527 non-null  object 
 13  RTN_PROCESS   38621 non-null   float64
 14  tempDATE      259527 non-null  object 
 15  LEAS_hour     259527 non-null  object 
dtypes: float64(3), int64(7), object(6)
memory usage: 33.7+ MB


#막차 시간 이후 자전거 현황

In [15]:
# 대여-반납 현황이 시간대별로 유사하기 때문에 대여 관련 자료만 추출한다.

bicycle_makcha_LEAS=bicycle_makcha[['LEAS_NO', 'LEAS_DATE', 'LEAS_STATION', 'LEAS_DEF_NO']]
bicycle_makcha_LEAS

,LEAS_NO,LEAS_DATE,LEAS_STATION,LEAS_DEF_NO
0,15945541,2017-01-01 00:00:41,213,18
1,15945542,2017-01-01 00:01:03,231,4
2,15945543,2017-01-01 00:01:50,119,14
3,15945544,2017-01-01 00:02:09,121,17
4,15945545,2017-01-01 00:03:32,320,29
...,...,...,...,...
3141701,21716337,2019-12-31 03:12:34,307,13
3141702,21716339,2019-12-31 03:16:21,238,2
3141703,21716342,2019-12-31 03:22:43,210,10
3141704,21716346,2019-12-31 03:28:54,306,14


In [16]:
# 결측값 확인
bicycle_makcha_LEAS.isnull().sum()

LEAS_NO         0
LEAS_DATE       0
LEAS_STATION    0
LEAS_DEF_NO     0
dtype: int64

In [17]:
#LEAS_STATION의 대여 횟수
LEAS_counts=bicycle_makcha_LEAS['LEAS_STATION'].value_counts()

In [19]:
# 막차 시간대 대여횟수를 하나의 데이터프레임으로 변형
hour_0to3 = LEAS_counts.rename_axis('LEAS_STATION').reset_index(name='counts')
hour_0to3

,LEAS_STATION,counts
0,210,15446
1,260,6489
2,259,5992
3,306,5298
4,342,4951
...,...,...
151,351,37
152,256,37
153,992,33
154,263,16


In [26]:
# 해당 스테이션별 위도 경도 입히기

bicycle_station=pd.read_csv('02.자전거스테이션.csv')

In [27]:
hour_0to3['위도']=0
hour_0to3['경도']=0

for i in range(len(hour_0to3['LEAS_STATION'])):
    for j in range(len(bicycle_station['Station_ID'])):
        if hour_0to3['LEAS_STATION'].iloc[i]==bicycle_station['Station_ID'].iloc[j]:
            hour_0to3['위도'].iloc[i]=bicycle_station['위도'].iloc[j]
            hour_0to3['경도'].iloc[i]=bicycle_station['경도'].iloc[j]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [28]:
hour_0to3.head()

,LEAS_STATION,counts,위도,경도
0,210,15446,37.663673,126.767099
1,260,6489,37.662030,126.768952
2,259,5992,37.656722,126.771620
3,306,5298,37.675838,126.747974
4,342,4951,37.677995,126.766402
